# KRX 연간 재무제표 데이터 수집

In [1]:
import numpy as np
import pandas as pd
import nfn_crawring as nf
import gbq_service as gbq

In [2]:
df_krx = gbq.get_krx_list()
df_krx = df_krx.sort_values('market_cap', ascending=False).reset_index(drop=True)
df_krx

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,005930,삼성전자,1975-06-11,KOSPI,None,5969782550,444151821720000
1,000660,SK하이닉스,1996-12-26,KOSPI,None,728002365,98280319275000
2,373220,LG에너지솔루션,2022-01-27,KOSPI,None,234000000,88335000000000
3,207940,삼성바이오로직스,2016-11-10,KOSPI,None,71174000,56939200000000
4,005380,현대차,1974-06-28,KOSPI,None,211531506,41375562573600
...,...,...,...,...,...,...,...
2325,030790,비케이탑스,2010-11-02,KOSPI,None,21248145,19208323080
2326,071950,코아스,2005-08-04,KOSPI,None,30907210,15824491520
2327,078860,아이오케이,2006-06-27,KOSDAQ,중견기업부,4806323,15788771055
2328,008500,일정실업,1994-08-12,KOSPI,None,1200000,15480000000


In [3]:
# collection_list = df_krx[['stock_code', 'corp_name']].values[0:2]
collection_list = df_krx[['stock_code', 'corp_name']].values

In [4]:
df_gbq = pd.DataFrame()
df_exception = pd.DataFrame(columns=['stock_code', 'corp_name', 'err_msg'])

for index, corp in enumerate(collection_list):
    try:
        print('No. ', index)
        print(corp, '재무제표(연간) 데이터 크롤링 시작 ===>')
        # 연간 손익계산서
        df_is = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.IS, period=nf.Period.YEAR)
        df_is = gbq.convert_gbq_year(df_is, corp[0], corp[1], 'IS')
        
        # 연간 재무상태표
        df_bs = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.BS, period=nf.Period.YEAR)
        df_bs = gbq.convert_gbq_year(df_bs, corp[0], corp[1], 'BS')
        
        # 연간 현금흐름표
        df_cf = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.CF, period=nf.Period.YEAR)
        df_cf = gbq.convert_gbq_year(df_cf, corp[0], corp[1], 'CF')
        
        # 데이터 합치기
        df_gbq = pd.concat([df_gbq, df_is, df_bs, df_cf])
        
        print(corp, '재무제표(연간) 데이터 크롤링 종료 ===>\n\n')
        
    except Exception as e:
        err_msg = str(type(e)) + ': ' + str(e)
        print(err_msg)
        df_exception.loc[len(df_exception)] = [corp[0], corp[1], err_msg]

No.  0
['005930' '삼성전자'] 재무제표(연간) 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=0&finGubun=MAIN&frqTyp=0&cn=&encparam=N0ZVR3VBQk1Ud2gwczJUWmxmSjg0Zz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=1&finGubun=MAIN&frqTyp=0&cn=&encparam=Uy9oaER3Z3RZL3NtMjRHY1IvSHBuQT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=2&finGubun=MAIN&frqTyp=0&cn=&encparam=N2hhTXJHc2pJSFVWb21JYUVDQy90Zz09
['005930' '삼성전자'] 재무제표(연간) 데이터 크롤링 종료 ===>


No.  1
['000660' 'SK하이닉스'] 재무제표(연간) 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=0&finGubun=MAIN&frqTyp=0&cn=&encparam=VmRRZTRxaWk4WDdOczJBZjY1Y1ZsZz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=1&finGubun=MAIN&frqTyp=0&cn=&encparam=dnNIdTFjNnhFY1JjS21UWEh3bC9mdz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=2&finG

In [5]:
df_gbq.to_csv('./csv/fs_year_240202.csv')
df_gbq

,stock_code,corp_name,year,month,type,order,account_name,amount
id,,,,,,,,
005930_2018_12_IS_000,005930,삼성전자,2018,12,IS,0,매출액(수익),2.437714e+06
005930_2018_12_IS_001,005930,삼성전자,2018,12,IS,1,*내수,NaN
005930_2018_12_IS_002,005930,삼성전자,2018,12,IS,2,*수출,NaN
005930_2018_12_IS_003,005930,삼성전자,2018,12,IS,3,....제품매출액,NaN
005930_2018_12_IS_004,005930,삼성전자,2018,12,IS,4,....상품매출액,NaN
...,...,...,...,...,...,...,...,...
161570_2022_12_CF_307,161570,THE MIDONG,2022,12,CF,307,연결범위변동으로인한현금의증가,NaN
161570_2022_12_CF_308,161570,THE MIDONG,2022,12,CF,308,환율변동효과,-2.025646e-01
161570_2022_12_CF_309,161570,THE MIDONG,2022,12,CF,309,현금및현금성자산의증가,-3.290291e+01


In [6]:
gbq.load_to_fs_year(df_gbq)

LoadJob<project=hello-phase3, location=asia-northeast3, id=96c3cf53-9dcf-4dee-add7-8913161793ce> 
Uploaded:  9127786


In [7]:
df_exception

,stock_code,corp_name,err_msg
0,068400,SK렌터카,<class 'requests.exceptions.JSONDecodeError'>:...
